In [1]:
local = True
log = True
log_detail = False

# Imports

## Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import os
import sys
from tqdm import tqdm
sys.path.append(os.path.abspath('..'))
import importlib
import pickle

In [3]:
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

/Users/allison/Downloads/idsn544/math-misconceptions/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Data

In [5]:
if local:
    misconceptions = pd.read_csv('../kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv', index_col='MisconceptionId')
    train = pd.read_csv('../kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv')
    test = pd.read_csv('../kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv')
else:
    misconceptions = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv', index_col='MisconceptionId')
    train = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv')
    test = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv')
if log: print("(1) Imported data")

(1) Imported data


In [6]:
#########################
# Create train_melted 
#########################

# Define the identifier columns
id_cols = [
    'QuestionId', 'ConstructId', 'ConstructName', 
    'SubjectId', 'SubjectName', 'CorrectAnswer', 'QuestionText'
]

# Define the corresponding Answer options
answer_cols = ['AnswerAText', 'AnswerBText', 'AnswerCText', 'AnswerDText']
misconception_cols = ['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId']

# Melt Answer Text
text_melted = train.melt(
    id_vars=id_cols,
    value_vars=answer_cols,
    var_name='Attribute',
    value_name='AnswerText'
)

# Melt Misconception IDs
misconception_melted = train.melt(
    id_vars=id_cols,
    value_vars=misconception_cols,
    var_name='Attribute',
    value_name='MisconceptionId'
)

# Extract the option letter (A, B, C, D) and the attribute type
text_melted['AnswerOption'] = text_melted['Attribute'].str.extract(r'Answer([ABCD])Text')[0]
misconception_melted['AnswerOption'] = misconception_melted['Attribute'].str.extract(r'Misconception([ABCD])Id')[0]

# Drop the original 'Attribute' columns as they are no longer needed
text_melted.drop('Attribute', axis=1, inplace=True)
misconception_melted.drop('Attribute', axis=1, inplace=True)

# Merge the two melted DataFrames on id_vars and AnswerOption
train_melted = pd.merge(
    text_melted,
    misconception_melted,
    on=id_cols + ['AnswerOption'],
    how='left'
)

train_melted = train_melted.merge(misconceptions, left_on='MisconceptionId', right_index=True, how='left')
train_melted = train_melted[train_melted['CorrectAnswer'] != train_melted['AnswerOption']]

In [7]:
print(f"SubjectName nunique: {train_melted['SubjectName'].nunique()}")
print(f"ConstructName nunique: {train_melted['ConstructName'].nunique()}")
print(f"MisconceptionName nunique: {train_melted['MisconceptionName'].nunique()}")


SubjectName nunique: 163
ConstructName nunique: 757
MisconceptionName nunique: 1604


In [8]:
hierarchy_scm = {}
for subject in train_melted['SubjectName'].unique():
    subject_data = train_melted[train_melted['SubjectName'] == subject]
    constructs = subject_data['ConstructName'].dropna().unique()
    constructs_list = []
    for construct in constructs:
        misconceptions = subject_data[subject_data['ConstructName'] == construct]['MisconceptionName'].dropna().unique().tolist()
        constructs_list.extend(misconceptions)
    hierarchy_scm[subject] = constructs_list

# Classes

In [9]:
class Hierarchy:
    def __init__(self):
        self.parent_to_children = {}
        for subject, misconceptions in hierarchy_scm.items():
            self.parent_to_children[subject] = misconceptions
        self.child_to_parent = {child: parent for parent, children in self.parent_to_children.items() for child in children}
    
    def get_parent(self, child):
        return self.child_to_parent.get(child, None)
    
    def get_children(self, parent):
        return self.parent_to_children.get(parent, [])

In [10]:
class DataPreprocessor:
    def __init__(self, train_df, test_df, hierarchy):
        self.train_df = train_df
        self.test_df = test_df
        self.hierarchy = hierarchy
        self.label_encoders = {}
        self.child_label_encoders = {}
    
    def map_hierarchy(self):
        self.train_df['ParentCategory'] = self.train_df['MisconceptionName'].apply(self.hierarchy.get_parent)
        self.test_df['ParentCategory'] = None  # To be predicted later
    
    def encode_labels(self):
        # Encode parent categories
        le_parent = LabelEncoder()
        self.train_df['ParentCategoryEncoded'] = le_parent.fit_transform(self.train_df['ParentCategory'])
        self.label_encoders['parent'] = le_parent
        
        # Encode specific misconceptions per parent
        self.child_label_encoders = {}
        self.train_df['MisconceptionEncoded'] = -1  # Initialize with a placeholder
        
        for parent in self.hierarchy.parent_to_children.keys():
            misconception_subset = self.train_df[self.train_df['ParentCategory'] == parent]['MisconceptionName']
            le_child = LabelEncoder()
            encoded = le_child.fit_transform(misconception_subset)
            self.train_df.loc[self.train_df['ParentCategory'] == parent, 'MisconceptionEncoded'] = encoded
            self.child_label_encoders[parent] = le_child
        
    def preprocess(self):
        self.map_hierarchy()
        self.encode_labels()
        # Additional preprocessing steps like text cleaning can be added here
        return self.train_df, self.test_df

In [11]:
class HierarchicalClassifier:
    def __init__(self, parent_num_labels, child_num_labels, pretrained_model='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(pretrained_model)
        self.parent_model = BertForSequenceClassification.from_pretrained(pretrained_model, num_labels=parent_num_labels)
        self.child_models = nn.ModuleDict()
        self.pretrained_model = pretrained_model
    
    def add_child_model(self, parent_category, num_labels):
        # Convert parent category to valid dictionary key
        parent_key = str(parent_category).replace(" ", "_").replace("/", "_")
        if parent_key not in self.child_models:
            try:
                self.child_models[parent_key] = BertForSequenceClassification.from_pretrained(
                    self.pretrained_model, 
                    num_labels=num_labels
                )
                if log:
                    print(f"Created child model for {parent_category} with {num_labels} labels")
            except Exception as e:
                print(f"Error creating child model for {parent_category}: {str(e)}")
                raise
                
    def forward_parent(self, input_ids, attention_mask):
        return self.parent_model(input_ids=input_ids, attention_mask=attention_mask)
    
    def forward_child(self, parent_category, input_ids, attention_mask):
        child_model = self.child_models.get(parent_category, None)
        if child_model:
            return child_model(input_ids=input_ids, attention_mask=attention_mask)
        else:
            return None

In [12]:
class DistractorDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

def train_stage1(model, train_df, tokenizer, device, epochs=3):
    dataset = DistractorDataset(
        texts=train_df['AnswerText'].tolist(),
        labels=train_df['ParentCategoryEncoded'].tolist(),
        tokenizer=tokenizer
    )
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    optimizer = torch.optim.AdamW(model.parent_model.parameters(), lr=2e-5)
    model.parent_model.to(device)
    model.parent_model.train()
    
    for epoch in range(epochs):
        total_loss = 0.0
        total_correct = 0
        total_examples = 0
        
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model.parent_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            total_correct += (preds == labels).sum().item()
            total_examples += labels.size(0)
            
            loss.backward()
            optimizer.step()
        
        avg_loss = total_loss / len(dataloader)
        accuracy = total_correct / total_examples
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

def train_stage2(model, train_df, tokenizer, hierarchy, device, epochs=3):
    for parent in tqdm(hierarchy.parent_to_children.keys()):
        # Convert parent name to valid dictionary key right away
        parent_key = str(parent).replace(" ", "_").replace("/", "_")
        
        child_subset = train_df[train_df['ParentCategory'] == parent]
        if child_subset.empty:
            if log: print(f"Skipping {parent} - no data")
            continue
            
        # Get number of unique misconceptions for this parent
        num_labels = len(child_subset['MisconceptionEncoded'].unique())
        
        # Initialize the child model if it doesn't exist
        try:
            model.add_child_model(parent, num_labels)
            if log: print(f"Successfully added child model for {parent}")
        except Exception as e:
            print(f"Failed to add child model for {parent}: {e}")
            continue
        
        try:
            dataset = DistractorDataset(
                texts=child_subset['AnswerText'].tolist(),
                labels=child_subset['MisconceptionEncoded'].tolist(),
                tokenizer=tokenizer
            )
            dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
            
            # Now we can safely access the child model
            child_model = model.child_models[parent_key].to(device)
            optimizer = torch.optim.AdamW(child_model.parameters(), lr=2e-5)
            child_model.train()
            
            for epoch in range(epochs):
                total_loss = 0
                for batch in dataloader:
                    optimizer.zero_grad()
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)
                    
                    # Ensure labels are Long type
                    labels = labels.long()
                    
                    outputs = child_model(
                        input_ids=input_ids, 
                        attention_mask=attention_mask, 
                        labels=labels
                    )
                    
                    loss = outputs.loss
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                    
                avg_loss = total_loss / len(dataloader)
                if log:
                    print(f"Parent: {parent}, Epoch: {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")
        except Exception as e:
            print(f"Error training model for {parent}: {e}")
            continue

In [13]:
def predict(model, texts, hierarchy, tokenizer, label_encoders, device):
    dataset = DistractorDataset(
        texts=texts,
        labels=[0]*len(texts),  # Dummy labels
        tokenizer=tokenizer
    )
    dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
    model.parent_model.to(device)
    model.parent_model.eval()
    
    parent_preds = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model.parent_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            parent_preds.extend(preds)
    
    parent_labels = label_encoders['parent'].inverse_transform(parent_preds)
    specific_preds = []
    
    for i, parent in enumerate(parent_labels):
        child_model = model.child_models.get(parent, None)
        if child_model is None:
            specific_preds.append(None)
            continue
        child_model.to(device)
        child_model.eval()
        with torch.no_grad():
            encoding = tokenizer.encode_plus(
                texts[i],
                add_special_tokens=True,
                max_length=128,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)
            outputs = child_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=1).cpu().numpy()[0]
            # Use the specific encoder for the parent
            specific_misconception = label_encoders['child_label_encoders'][parent].inverse_transform([pred])[0]
            specific_preds.append(specific_misconception)
    
    return specific_preds

# Training

In [14]:
hierarchy = Hierarchy()

In [15]:
# Define paths to your data files
train_path = '../kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv'
test_path = '../kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv'

# Initialize DataPreprocessor
preprocessor = DataPreprocessor(train_df=train_melted, test_df=test, hierarchy=hierarchy)
train_df, test_df = preprocessor.preprocess()

In [16]:
# Determine the number of parent labels
parent_num_labels = train_df['ParentCategoryEncoded'].nunique()

# Initialize HierarchicalClassifier
model = HierarchicalClassifier(parent_num_labels=parent_num_labels, child_num_labels=0)  # child_num_labels handled per parent

/Users/allison/Downloads/idsn544/math-misconceptions/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using device: {device}')

Using device: cpu


In [18]:
# Train Stage 1: Parent Category Classification
train_stage1(model, train_df, model.tokenizer, device)

Epoch 1/3: 100%|██████████| 176/176 [11:57<00:00,  4.08s/it]


Epoch 1/3 - Loss: 4.4837, Accuracy: 0.2056


Epoch 2/3: 100%|██████████| 176/176 [11:21<00:00,  3.87s/it]


Epoch 2/3 - Loss: 4.1692, Accuracy: 0.2204


Epoch 3/3: 100%|██████████| 176/176 [12:10<00:00,  4.15s/it]

Epoch 3/3 - Loss: 3.9742, Accuracy: 0.2256


: 

In [19]:
# Train Stage 2: Specific Misconception Classification
train_stage2(model, train_df, model.tokenizer, hierarchy, device)

  0%|          | 0/163 [00:00<?, ?it/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Created child model for Simplifying Algebraic Fractions with 8 labels
Successfully added child model for Simplifying Algebraic Fractions
Parent: Simplifying Algebraic Fractions, Epoch: 1/3, Average Loss: 2.1719
Parent: Simplifying Algebraic Fractions, Epoch: 2/3, Average Loss: 2.1498


  1%|          | 1/163 [00:07<19:43,  7.31s/it]

Parent: Simplifying Algebraic Fractions, Epoch: 3/3, Average Loss: 2.0282


...

Created child model for Experimental Probability and Relative Frequency with 1 labels
Successfully added child model for Experimental Probability and Relative Frequency


 25%|██▍       | 40/163 [08:58<24:48, 12.10s/it]

Error training model for Experimental Probability and Relative Frequency: Found dtype Long but expected Float


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Created child model for Horizontal and Vertical Lines with 8 labels
Successfully added child model for Horizontal and Vertical Lines


In [ ]:
import os

# Create directory to save models
os.makedirs('models', exist_ok=True)

# Save parent model
model.parent_model.save_pretrained('models/parent_model')

# Save child models
for parent, child_model in model.child_models.items():
    model.child_models[parent].save_pretrained(f'models/child_model_{parent}')

In [ ]:
from sklearn.metrics import classification_report

def evaluate_stage1(model, train_df, tokenizer, device):
    dataset = DistractorDataset(
        texts=train_df['DistractorText'].tolist(),
        labels=train_df['ParentCategoryEncoded'].tolist(),
        tokenizer=tokenizer
    )
    dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
    model.parent_model.to(device)
    model.parent_model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model.parent_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    print(classification_report(all_labels, all_preds, target_names=preprocessor.label_encoders['parent'].classes_))

# Evaluate Stage 1
evaluate_stage1(model, train_df, model.tokenizer, device)

In [ ]:
def evaluate_stage2(model, train_df, tokenizer, hierarchy, device):
    for parent in hierarchy.parent_to_children.keys():
        child_subset = train_df[train_df['ParentCategory'] == parent]
        if child_subset.empty:
            continue
        
        child_model = model.child_models.get(parent, None)
        if child_model is None:
            continue
        
        child_model.to(device)
        child_model.eval()
        
        dataset = DistractorDataset(
            texts=child_subset['DistractorText'].tolist(),
            labels=child_subset['MisconceptionEncoded'].tolist(),
            tokenizer=tokenizer
        )
        dataloader = DataLoader(dataset, batch_size=16, shuffle=False)
        
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = child_model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
        
        print(f"Classification Report for Parent Category: {parent}")
        print(classification_report(all_labels, all_preds, target_names=preprocessor.label_encoders['child'].classes_))
        print("\n")

# Evaluate Stage 2
evaluate_stage2(model, train_df, model.tokenizer, hierarchy, device)